In [ ]:
import sys
import os
import glob
import pandas as pd

In [ ]:
file_cluster = glob.glob('../data/clean/cp_hourly/clusters/*.csv')

In [ ]:
generation = pd.read_csv('../generation_test.tab', sep='\t', index_col=0)
generation.head()

In [ ]:
generation_wind = generation.loc[generation['gen_tech'].isin(['wind', 'solarpv', 'solarpv_distributed'])]
generation_wind.head()

In [ ]:
rens = []
for index, row in generation_wind.iterrows():
    lz = row['gen_load_zone'].split('-')[0]
    gen = row['gen_tech'].split('_')[0]
    proj = 'proj1'
    lz = "{:d}".format(int(lz))
    file = [s for s in file_cluster if lz in s and gen in s and proj in s][0]
    tmp = pd.read_csv(file, index_col=0, parse_dates=True)
    tmp.loc[:, 'project_name'] = index
    if gen == 'solarpv':
        tmp.loc[:, 'capacity_factor'] = tmp.loc[:, 'cf']
    else:
        tmp.loc[:, 'capacity_factor'] = tmp.loc[:,'generation']/2000
    tmp.loc[:, 'time'] = tmp.index.strftime('%m-%d %H:%M:%S')
    tmp.loc[tmp.index.minute == 30, 'time'] = (tmp.index - pd.Timedelta(minutes=30) ).strftime('%m-%d %H:%M:%S')
    tmp.index.names = ['datetime']
    rens.append(tmp)

rens = pd.concat(rens)

In [ ]:
parent_path = os.path.dirname(os.path.dirname('..'))
file_path = os.path.join('../', 'data/clean/SWITCH/')
filename = 'ren-all2.csv'
ren_cap_data = pd.read_csv(os.path.join(file_path, filename), index_col=0,
                           parse_dates=['datetime'])
ren_cap_data['project_name'] = ren_cap_data['GENERATION_PROJECT']
print (len(ren_cap_data.project_name.unique()))
ren_cap_data.info()

In [ ]:
ren_cap_data = ren_cap_data.loc[~ren_cap_data.project_name.isnull()]
print (len(ren_cap_data.project_name.unique()))

In [ ]:
merged = pd.concat([ren_cap_data, rens])

In [ ]:
assert len(merged.project_name.unique()) == (len(rens.project_name.unique()) + len(ren_cap_data.GENERATION_PROJECT.unique()))

In [ ]:
parent_path = os.path.dirname(os.path.dirname('..'))
file_path = os.path.join('../', 'data/clean/SWITCH/')
filename = 'ren-all5.csv'
merged.to_csv(os.path.join(file_path, filename))